In [3]:
import os

In [2]:
%pwd

'd:\\DS_project\\Titanic_project\\research'

In [4]:
os.chdir('../')

In [4]:
%pwd

'd:\\DS_project\\Titanic_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox


In [5]:
from Titanic_classifier.constant import *
from Titanic_classifier.utils.common import read_yaml,create_directories
from Titanic_classifier import logging
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import Tuple

In [7]:
@dataclass
class DataIngestionConfig:
    train_data_path:str=os.path.join('artifacts','train.csv')
    test_data_path:str=os.path.join('artifacts','test.csv')
    raw_data_path:str=os.path.join('artifacts','data.csv')

class DataIngestion:
    def __init__(self):
        self.ingestion_config = DataIngestionConfig()

    def initiate_data_ingestion(self) -> Tuple[str, str, str]:
        logging.info('Entered the data ingestion method or component')
        try:
            df = pd.read_csv(r'research\Data\Titanic.csv')
            logging.info('Read the dataset as dataframe')
            os.makedirs(os.path.dirname(self.ingestion_config.train_data_path),exist_ok=True)
            df.to_csv(self.ingestion_config.raw_data_path,index=False,header=True)
            logging.info('Train test split initiated')
            train_set,test_set = train_test_split(df,test_size=0.2,random_state = 42)
            train_set.to_csv(self.ingestion_config.train_data_path,index= False,header=True)
            test_set.to_csv(self.ingestion_config.test_data_path,index =False , header = True)
            logging.info('Ingestion completed')
            return (
                self.ingestion_config.train_data_path,
                self.ingestion_config.test_data_path,
                self.ingestion_config.raw_data_path
            )
        except Exception as e:
            logging.error(f'Error during data ingestion: {e}')
            return "", "", ""


In [8]:
try:
    obj=DataIngestion()
    train_data,test_data,df=obj.initiate_data_ingestion()
except:
    pass

[2024-03-13 00:13:56,079: INFO: 3837923224: Entered the data ingestion method or component]
[2024-03-13 00:13:56,084: INFO: 3837923224: Read the dataset as dataframe]
[2024-03-13 00:13:56,097: INFO: 3837923224: Train test split initiated]
[2024-03-13 00:13:56,102: INFO: 3837923224: Ingestion completed]


In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [5]:
from Titanic_classifier.constant import *
from Titanic_classifier.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from Titanic_classifier import logging 
from Titanic_classifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logging.info(f"{filename} downloaded! with following info: \n{headers}")
            else:
                logging.info(f"File already exists of size: {os.path.getsize(self.config.local_data_file)} bytes")
        except Exception as e:
            logging.error(f"An error occurred during file download: {e}")

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
%pwd

'd:\\DS_project\\Titanic_project'

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-13 17:46:50,507: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-13 17:46:50,508: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-13 17:46:50,510: INFO: common: created directory at: artifacts]
[2024-03-13 17:46:50,511: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-13 17:46:51,290: INFO: 524653392: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 22508
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "05238c4df7751860f3cfa2ad4e8079fc9f329c86ba2e238dc0e5085c44bd4cf8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C16E:50422:1BA94B:217AA0:65F18397
Accept-Ranges: bytes
Date: Wed, 13 Mar 2024 10:46:50 GMT
Via: 1.1 varnish
X-Served-By: cache-bkk2310031-BKK
X-Cache: H